In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk import word_tokenize

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
import os 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
import numpy as np
from sklearn.decomposition import PCA
import torch as t
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

from string import punctuation
import string
from collections import Counter
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn as nn
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import bs4
from bs4 import BeautifulSoup
import re
from torch.autograd import Variable

import matplotlib.pyplot as plt
import pickle
from datetime import datetime

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
data_path = "/content/drive/My Drive/NLU/Labs/data/"
df =pd.read_csv(data_path + 'all-data.csv' , header=None , encoding='ISO-8859-1')
df.columns =['sentiment' ,'statement']
df.head()

,sentiment,statement
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [4]:
class CleanText:
    def __init__(self,review):
        self.review=review.lower()
    
    def remove_URL(self):
        url = re.compile(r'https?://\S+|www\.\S+')
    
        return url.sub('', self.review)
    
    def remove_html(self):
        html = re.compile(r'<.*?>')
    
        return html.sub('', self.review)
    
    def remove_not_ASCII(self):
        
        review = ''.join([word for word in self.review if word in string.printable])
        return review
    
    def remove_emoji(self):
        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
        return emoji_pattern.sub(r'', self.review)
    
    
    def remove_punct(self):
        table = str.maketrans('', '', string.punctuation)
    
        return self.review.translate(table)
    
    def remove_numbers(self):

#         t = ''.join([i for i in self.review if not i.isdigit()])
        num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
        return num.sub(r'NUMBER', self.review)

#         return t
    
    
    def review_to_words(self):
        nltk.download("stopwords", quiet=True)
        stemmer = PorterStemmer()
        
        words = BeautifulSoup(self.review, "html.parser").get_text() # Remove HTML tags
#         words = re.sub(r"[^a-zA-Z0-9]", " ", self.review.lower()) # Convert to lower case
        
#         words=self.review.lower()
        words=self.review.split() # Split string into words
        words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
        words = [PorterStemmer().stem(w) for w in words] # stem
        
        return words
    
    def all_func(self):
        self.review=self.remove_URL()
        self.review=self.remove_html()
        self.review=self.remove_not_ASCII()
        self.review=self.remove_emoji()
        self.review=self.remove_punct()
        self.review=self.remove_numbers()
        self.review=self.review_to_words()
        
        return self.review

In [5]:
df['statement']=df['statement'].apply(lambda x: CleanText(x).all_func())

In [6]:
df.head()

,sentiment,statement
0,neutral,"[accord, gran, compani, plan, move, product, r..."
1,neutral,"[technopoli, plan, develop, stage, area, less,..."
2,negative,"[intern, electron, industri, compani, elcoteq,..."
3,positive,"[new, product, plant, compani, would, increas,..."
4,positive,"[accord, compani, updat, strategi, year, numbe..."


In [7]:
df['len'] = df['statement'].apply(lambda s : len(s))

In [8]:
df.head()

,sentiment,statement,len
0,neutral,"[accord, gran, compani, plan, move, product, r...",10
1,neutral,"[technopoli, plan, develop, stage, area, less,...",18
2,negative,"[intern, electron, industri, compani, elcoteq,...",21
3,positive,"[new, product, plant, compani, would, increas,...",20
4,positive,"[accord, compani, updat, strategi, year, numbe...",22


In [9]:

X_train, X_test, y_train, y_test = train_test_split(df['statement'], df['sentiment'],
                                                    stratify=df['sentiment'], 
                                                    test_size=0.2)

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train.values)
y_test = le.transform(y_test.values)
print(len(y_train))
print(len(y_test))

3876
970


In [11]:
def convert_and_pad(word_dict, sentence, pad=46):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad

    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:      
            working_sentence[word_index+pad-len(sentence)] = word_dict[word]                      
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=46):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)


def words_to_numbers(x_train):
    words=[w for s in x_train for w in s]
    counts=Counter(words)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word:ii for ii,word in enumerate(vocab,1)}
    
    return vocab_to_int

def words_as_int(vocab_to_int,x_train):
    news_int = []
    x=0
    for review in x_train:
        x+=1
        news_int.append([vocab_to_int[word] for word in review])
        
    return news_int

In [12]:
vocab_to_int_X_train=words_to_numbers(X_train)

In [13]:
words_as_int_X_train=words_as_int(vocab_to_int_X_train,X_train)
len(words_as_int_X_train)

3876

In [14]:
# Max Len of Statement
news_len_train = Counter([len(x) for x in words_as_int_X_train])
print("Zero-length reviews: {}".format(news_len_train[0]))
print("Maximum review length: {}".format(max(news_len_train)))

Zero-length reviews: 0
Maximum review length: 46


In [15]:
padded_X_train,_=convert_and_pad_data(vocab_to_int_X_train, X_train, pad=46)
padded_X_train=padded_X_train.astype('int32')
padded_X_test,_=convert_and_pad_data(vocab_to_int_X_train, X_test, pad=46)
padded_X_test=padded_X_test.astype('int32')

GloVe Emmbedding

In [16]:
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 46 # max number of words in a question to use 750
batch_size = 46 # how many samples to process at once
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits
SEED = 10
debug = 0

In [17]:
def load_glove(word_index):
    EMBEDDING_FILE = data_path +'/glove6B/glove.6B.300d.txt'  
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [18]:
embedding_matrix = load_glove(vocab_to_int_X_train)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
embedding_matrix.shape

(6407, 300)

GloVe +BiLSTM Model

In [20]:
 class GloVe_BiLSTM_Model(nn.Module):   
    def __init__(self):
        super(GloVe_BiLSTM_Model, self).__init__()
        self.hidden_size = 64
        drp = 0.1
        n_classes = len(le.classes_)
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(self.hidden_size*4 , 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, n_classes)


    def forward(self, x):
        #rint(x.size())
        h_embedding = self.embedding(x)
        #_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))
        h_lstm, _ = self.lstm(h_embedding)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out

Model Training

In [41]:
n_epochs = 40
model = GloVe_BiLSTM_Model()
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0014)
model.cuda()

# Load train and test in CUDA Memory
x_train = torch.tensor(padded_X_train, dtype=torch.long).cuda()
y_train = torch.tensor(y_train, dtype=torch.long).cuda()
x_cv = torch.tensor(padded_X_test, dtype=torch.long).cuda()
y_cv = torch.tensor(y_test, dtype=torch.long).cuda()

# Create Torch datasets
train = torch.utils.data.TensorDataset(x_train, y_train)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)

# Create Data Loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

train_loss = []
valid_loss = []

for epoch in range(n_epochs):
    start_time = time.time()
    # Set model to train configuration
    model.train()
    avg_loss = 0.  
    for i, (x_batch, y_batch) in enumerate(train_loader):
        # Predict/Forward Pass
        y_pred = model(x_batch)
        # Compute loss
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
    
    # Set model to validation configuration -Doesn't get trained here
    model.eval()        
    avg_val_loss = 0.
    val_preds = np.zeros((len(x_cv),len(le.classes_)))
    
    for i, (x_batch, y_batch) in enumerate(valid_loader):
        y_pred = model(x_batch).detach()
        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        # keep/store predictions
        val_preds[i * batch_size:(i+1) * batch_size] =F.softmax(y_pred).cpu().numpy()
    
    # Check Accuracy
    val_accuracy = sum(val_preds.argmax(axis=1)==y_test)/len(y_test)
    train_loss.append(avg_loss)
    valid_loss.append(avg_val_loss)
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t val_acc={:.4f}  \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1/40 	 loss=40.2249 	 val_loss=34.7702  	 val_acc=0.6680  	 time=0.49s
Epoch 2/40 	 loss=32.8989 	 val_loss=30.3965  	 val_acc=0.6979  	 time=0.48s
Epoch 3/40 	 loss=24.6473 	 val_loss=28.0269  	 val_acc=0.7371  	 time=0.48s
Epoch 4/40 	 loss=17.6397 	 val_loss=28.4694  	 val_acc=0.7443  	 time=0.47s
Epoch 5/40 	 loss=11.1647 	 val_loss=31.7876  	 val_acc=0.7247  	 time=0.47s
Epoch 6/40 	 loss=7.3149 	 val_loss=39.7534  	 val_acc=0.7464  	 time=0.47s
Epoch 7/40 	 loss=4.4015 	 val_loss=37.8495  	 val_acc=0.7227  	 time=0.47s
Epoch 8/40 	 loss=3.5816 	 val_loss=43.8226  	 val_acc=0.7412  	 time=0.47s
Epoch 9/40 	 loss=2.7823 	 val_loss=46.1452  	 val_acc=0.7526  	 time=0.47s
Epoch 10/40 	 loss=1.9170 	 val_loss=47.7652  	 val_acc=0.7309  	 time=0.47s
Epoch 11/40 	 loss=1.6311 	 val_loss=46.9681  	 val_acc=0.7392  	 time=0.47s
Epoch 12/40 	 loss=1.3525 	 val_loss=53.6669  	 val_acc=0.7021  	 time=0.47s
Epoch 13/40 	 loss=1.3970 	 val_loss=50.2175  	 val_acc=0.7320  	 time=0.49s
Epo

In [42]:

torch.save(model.state_dict(), 'glove_biLSTM_40.pt')

In [43]:
model.load_state_dict(torch.load('glove_biLSTM_40.pt'))
model.eval()

GloVe_BiLSTM_Model(
  (embedding): Embedding(120000, 300)
  (lstm): LSTM(300, 64, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=256, out_features=64, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (out): Linear(in_features=64, out_features=3, bias=True)
)

In [44]:
from sklearn.metrics import classification_report, precision_recall_curve

y = [le.classes_[x] for x in y_test]
y_pred = [le.classes_[x] for x in val_preds.argmax(axis=1)]

In [45]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

    negative       0.55      0.74      0.63       121
     neutral       0.82      0.81      0.82       576
    positive       0.70      0.60      0.65       273

    accuracy                           0.75       970
   macro avg       0.69      0.72      0.70       970
weighted avg       0.75      0.75      0.75       970

